# Convolutional Neural Network

### Importing the libraries

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [11]:
tf.__version__

'2.18.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255, # feature scaling to each of the pixel
    shear_range=0.2, # these 3 lines are for transformation
    zoom_range=0.2,
    horizontal_flip=True
    )

training_set = train_datagen.flow_from_directory(
        'datasets/cnn-dataset/training_set',  # this is the target directory
        target_size=(64, 64),  # all images will be resized to 64x64
        batch_size=32,
        class_mode='binary' # since we use binary_crossentropy loss, we need binary labels
    )  


Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [24]:
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

test_set = test_datagen.flow_from_directory(
        'datasets/cnn-dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary'
    )

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [15]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [16]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu',input_shape=[64,64,3]))

c:\Users\xnoor\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [17]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [18]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [19]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [20]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [21]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [22]:
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [26]:
cnn.fit(x = training_set, validation_data=test_set, epochs=25)

c:\Users\xnoor\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 635ms/step - accuracy: 0.5398 - loss: 0.7083

c:\Users\xnoor\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


250/250 ━━━━━━━━━━━━━━━━━━━━ 197s 785ms/step - accuracy: 0.5399 - loss: 0.7082 - val_accuracy: 0.6815 - val_loss: 0.6272
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 28s 110ms/step - accuracy: 0.6597 - loss: 0.6230 - val_accuracy: 0.7010 - val_loss: 0.5784
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 19s 76ms/step - accuracy: 0.6921 - loss: 0.5745 - val_accuracy: 0.7325 - val_loss: 0.5406
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 20s 79ms/step - accuracy: 0.7232 - loss: 0.5421 - val_accuracy: 0.7500 - val_loss: 0.5143
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 21s 85ms/step - accuracy: 0.7458 - loss: 0.5121 - val_accuracy: 0.7635 - val_loss: 0.4981
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 20s 79ms/step - accuracy: 0.7499 - loss: 0.5038 - val_accuracy: 0.7705 - val_loss: 0.4871
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 21s 83ms/step - accuracy: 0.7631 - loss: 0.4722 - val_accuracy: 0.7540 - val_loss: 0.5137
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 23s 93ms/step - accuracy: 0.7658 - loss: 0.4749 - val_accu

## Part 4 - Making a single prediction

In [45]:
import numpy as np
from keras.preprocessing import image

test_image = image.load_img('datasets/cnn-dataset/single_prediction/pred5.jpg', target_size=(64,64))
test_image = image.img_to_array(test_image)  
test_image = np.expand_dims(test_image, axis=0) # dimension of the batch

result = cnn.predict(test_image)
training_set.class_indices # encoding 1 = dog , 0 = cat
if result[0][0] == 1: # batch then single prediction
    prediction = 'dog'
else: 
    prediction = 'cat'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


In [46]:
print(prediction)

cat
